
# 01\_GenAI\_Tracing\_Tokens\_Latency\_UI

## 🎯 Goal

Make every request **traceable, measurable, and cheap**—so you can debug fast and keep SLOs.

---

## 🧵 Tracing (end-to-end)

* 🪪 **IDs**: `trace_id`, `span_id`, `parent_span_id`, `request_id`, `user_id` (hash/anon).
* 🧭 **Spans**: `preproc` → `retrieval` → `rerank` → `llm_call` → `postproc` → `safety`.
* 🧷 **Context**: attach run info (experiment, run\_id, model/version, prompt\_id).
* 🧯 **Payload hygiene**: log **metadata & hashes**, not raw PII; store doc **IDs**, not full text.
* 🔌 **Hooks**: instrument SDK calls (HTTP, DB, vector store) to auto-create spans.

---

## 🔤 Tokens & 💸 Cost

* 📥 `tokens_in` (prompt) | 📤 `tokens_out` (completion) | ♻️ `tokens_cached` (if provider supports).
* 🧮 **Cost**: `cost_usd = (tokens_in/1000 * price_in) + (tokens_out/1000 * price_out)`.
* 🧩 Break down by **segment**: system, instructions, few-shot, context, user msg.
* 📊 Track **avg, p95** tokens and **context\_use\_rate** (did the answer cite retrieved docs?).
* 🧱 Guardrails: **per-request** and **daily** budget caps; **truncate** long context.

---

## ⏱️ Latency (SLO-driven)

* 🧩 **Breakdown**: queue → retrieval → rerank → LLM → postproc → safety.
* 🎯 **Targets**: p50 ≤ 500 ms, p95 ≤ 1200 ms (tune per app).
* ⚙️ Tactics: **caching**, **streaming**, **batching**, **early stop**, **smaller context**.
* 🧰 Resilience: timeouts, retries w/ backoff, fallbacks (cheaper model or shorter prompt).

---

## 🪟 UI (what to build)

* 📈 **Dashboards**: latency (p50/p95), error\_rate, tokens, cost, guardrail blocks.
* 🔎 **Trace viewer**: waterfall per request; click into slow spans.
* 🧪 **Compare runs**: MLflow filters like
  `params.llm.model='gpt-4o' AND metrics.latency_ms_p95 < 1200 AND metrics.cost_usd < 0.002`
* 🧭 **Saved views**: “High-cost prompts”, “Slow queries”, “Safety blocks spike”.
* 🧩 **Top offenders** tables: heaviest prompts, largest contexts, slowest docs.

---

## 🔔 Alerts (pager-worthy)

* ⏱️ **p95 latency** > SLO (5 min)
* ❌ **error\_rate** > 2%
* 💸 **cost/min** > expected band
* 🧯 **safety\_block\_rate** spike vs baseline

---

## 🔐 Privacy & governance

* 🧽 **Redact** PII before logging; hash user IDs.
* 🗂️ **Retention**: short for raw traces, longer for **aggregates**.
* 🧾 Link traces ↔ **model version** & **prompt version** for audits.

---

## ✅ Minimal fields to log (per request)

* IDs: `trace_id`, `request_id`, `model_id`, `model_version`, `prompt_id`, `dataset_id?`
* Metrics: `latency_ms_total`, `latency_ms_llm`, `tokens_in/out`, `cost_usd`, `error?`
* RAG: `retriever_k`, `hit_at_k`, `context_precision`, `context_use_rate`
* Safety: `toxicity_flag`, `pii_flag`, `block_reason`
* Artifacts (optional): trace JSONL, sampled prompt render (sanitized)

---

## 🧪 Quick setup checklist

* [ ] Add **middleware** to stamp IDs + timers
* [ ] Instrument **retriever + LLM** calls as spans
* [ ] Compute **token & cost** server-side (don’t trust client)
* [ ] Ship **metrics** to your TSDB + **runs** to MLflow
* [ ] Create **saved UI views** + **alerts**
* [ ] Validate with a **smoke trace** on each deploy

---

## 🗣️ One-liner

**“Trace every step, count every token, watch p95—not just averages—and surface it all in a clean dashboard.”**
